# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers sentence-transformers torchmetrics gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import warnings
warnings.filterwarnings("ignore")

# Import

In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from copy import deepcopy
from torchmetrics import Precision
import torch
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm
import json

# Dataset

In [6]:
evaluate_data = pd.read_csv("/content/drive/MyDrive/Achatbot/data/test_case/processed_test_case.csv")
evaluate_data.rename(columns={'Unnamed: 0': "ID", "Câu lệnh nhập vào": "Câu lệnh sinh ra"}, inplace=True)
evaluate_data.head()

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn,Output,Dự đoán
0,0,Mở cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng
1,1,cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng
2,2,cảnh báo KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng
3,3,cảnh báo tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng
4,4,cảnh báo kế hoạch tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng


# Model

In [7]:
model = SentenceTransformer("/content/drive/MyDrive/Achatbot/output/model")
model.load("/content/drive/MyDrive/Achatbot/output/model")
model.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

# Necessary def

In [8]:
def cosinesimilarity(vector1, vector2):
    cosine = np.dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    return cosine

def encode_all_and_match(input, db_in):
    print("Input:", input)
    embedding = model.encode(input)
    df = pd.DataFrame(list(zip(db_in, [0]*len(db_in))), columns=["Câu lệnh có sẵn", "Điểm"])
    for i, func in tqdm(enumerate(db_in)):
        embedding2 = model.encode(func)
        df['Điểm'].loc[i] = cosinesimilarity(embedding, embedding2)
    df.sort_values(by=['Điểm'], inplace=True, ascending=False)
    return df

def encode_database(db_in):
    df = pd.DataFrame(list(zip(db_in, [[]]*len(db_in))), columns=["Câu lệnh có sẵn", "Embedding"])
    for i, func in tqdm(enumerate(db_in)):
        embedding2 = model.encode(func)
        df['Embedding'].loc[i] = embedding2
    else:
        print()
        print("Encode database successfully")
    return df

def encode_input_and_match(input_in, db_dff):
    print("Input:", input_in)
    embedding = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in tqdm(enumerate(db_in)):
        embedding2 = db_df_in['Embedding'].loc[i]
        scores.append(cosinesimilarity(embedding, embedding2))
    else:
        print()
        print("Matching process finished.")
    db_df_in["Điểm"] = scores
    db_df_in.sort_values(by=['Điểm'], inplace=True, ascending=False)
    return db_df_in[["Câu lệnh có sẵn", "Điểm"]]

def encode_input_and_return_output(input_in, db_dff):
    embed1 = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in enumerate(db_in):
        embed2 = db_df_in['Embedding'].loc[i]
        scores.append(cosinesimilarity(embed1, embed2))
    db_df_in["Điểm"] = scores
    return db_df_in["Câu lệnh có sẵn"].loc[db_df_in['Điểm'].idxmax()]

def encode_input_and_return_top_n(input_in, db_dff, top_k):
    embed1 = model.encode(input_in)
    scores = []
    db_df_in = deepcopy(db_dff)
    db_in = list(set(db_df_in['Câu lệnh có sẵn'].tolist()))
    for i, func in enumerate(db_in):
        embed2 = db_df_in['Embedding'].loc[i]
        scores.append(round(cosinesimilarity(embed1, embed2), 3))
    db_df_in["Điểm"] = scores
    db_df_in.sort_values(by=['Điểm'], inplace=True, ascending=False)
    ids = db_df_in[:top_k].index.tolist()
    output = {db_df_in['Câu lệnh có sẵn'][i]: round(db_df_in['Điểm'][i].item(), 2) for i in ids}
    return output

## Encode database

In [24]:
with open('/content/drive/MyDrive/Achatbot/data/new2oldmatch.json', 'r') as openfile:
    new2oldmatch = json.load(openfile)
    old2newmatch = {i.strip(): j.strip() for j, i in new2oldmatch.items()}

evaluate_data['Dự đoán'] = evaluate_data['Dự đoán'].apply(lambda x: old2newmatch[x])
evaluate_data['Câu lệnh có sẵn'] = evaluate_data['Câu lệnh có sẵn'].apply(lambda x: old2newmatch[x.strip()])

KeyError: ignored

In [23]:
old2newmatch

{'Mở báo cáo KPI chấm điểm': 'Mở báo cáo KPI chấm điểm',
 'Mở quản lý thư mục': 'Mở quản lý thư mục',
 'Mở báo cáo dữ liệu tin nhắn kinh doanh': 'Mở báo cáo dữ liệu tin nhắn kinh doanh',
 'Mở hướng dẫn sử dụng': 'Mở hướng dẫn sử dụng',
 'Mở quản lý nhóm đơn vị': 'Mở quản lý nhóm đơn vị',
 'Mở quản lý SMS/Email': 'Mở quản lý SMS/Email',
 'Mở màn hình trình chiếu báo cáo tuần': 'Mở màn hình trình chiếu báo cáo tuần',
 'Mở profile': 'Mở profile',
 'Mở quản lý chu kỳ mặc định': 'Mở quản lý chu kỳ mặc định',
 'Mở BC_CSKH': 'Mở báo cáo chăm sóc khách hàng',
 'Mở cảnh báo nhập KH quý': 'Mở cảnh báo nhập khách hàng quý',
 'Mở quản lý hướng dẫn sử dụng': 'Mở quản lý hướng dẫn sử dụng',
 'Mở tra cứu chỉ tiêu': 'Mở tra cứu chỉ tiêu',
 'Mở quản lý cấu hình người nhận cảnh báo': 'Mở quản lý cấu hình người nhận cảnh báo',
 'Mở màn hình trình chiếu báo cáo ngày': 'Mở màn hình trình chiếu báo cáo ngày',
 'Mở quản lý chu kỳ import tài chính, kinh doanh': 'Mở quản lý chu kỳ import tài chính, kinh doanh'

In [12]:
database = [cmd.lower() for cmd in new2oldmatch.keys()]
db_df = encode_database(database)

55it [00:09,  5.66it/s]


Encode database successfully


# Dự đoán trên tập dữ liệu

In [13]:
datadata = evaluate_data
results = deepcopy(datadata)
results['Dự đoán myself'] = [0]*len(results)
for i, ints in tqdm(enumerate(results['Câu lệnh sinh ra'])):
    output = encode_input_and_return_output(ints, db_df)
    results['Dự đoán myself'].loc[i] = output

290it [00:03, 79.11it/s]


In [14]:
results

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn,Output,Dự đoán,Dự đoán myself
0,0,Mở cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
1,1,cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
2,2,cảnh báo KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
3,3,cảnh báo tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
4,4,cảnh báo kế hoạch tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
...,...,...,...,...,...,...
285,285,bc kpi,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở dashboard
286,286,BC KPI,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
287,287,BC KPI CĐ,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
288,288,MỞ BÁO CÁO KPI CHẤM ĐIỂM,Mở báo cáo KPI chấm điểm,0,Mở profile,mở profile


In [15]:
results.drop(columns=["ID"])

,Câu lệnh sinh ra,Câu lệnh có sẵn,Output,Dự đoán,Dự đoán myself
0,Mở cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
1,cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
2,cảnh báo KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
3,cảnh báo tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
4,cảnh báo kế hoạch tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
...,...,...,...,...,...
285,bc kpi,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở dashboard
286,BC KPI,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
287,BC KPI CĐ,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
288,MỞ BÁO CÁO KPI CHẤM ĐIỂM,Mở báo cáo KPI chấm điểm,0,Mở profile,mở profile


In [16]:
errors = results[results['Dự đoán']!=results['Dự đoán myself']]
errors

,ID,Câu lệnh sinh ra,Câu lệnh có sẵn,Output,Dự đoán,Dự đoán myself
0,0,Mở cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
1,1,cảnh báo nhập KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
2,2,cảnh báo KH tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
3,3,cảnh báo tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
4,4,cảnh báo kế hoạch tháng,Mở cảnh báo nhập KH tháng,1,Mở cảnh báo nhập KH tháng,mở cảnh báo nhập kế hoạch tháng
...,...,...,...,...,...,...
285,285,bc kpi,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở dashboard
286,286,BC KPI,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
287,287,BC KPI CĐ,Mở báo cáo KPI chấm điểm,0,Mở BC_CSKH,mở báo cáo chăm sóc khách hàng
288,288,MỞ BÁO CÁO KPI CHẤM ĐIỂM,Mở báo cáo KPI chấm điểm,0,Mở profile,mở profile


In [17]:
errors['Câu lệnh có sẵn'].value_counts()

Mở cảnh báo nhập KH quý                               20
Mở báo cáo chốt tháng (báo cáo chốt dữ liệu (BCL))    19
Mở báo cáo tỷ giá mới (đang phát triển)               18
Mở báo cáo chi tiết n2 mới                            18
Mở cảnh báo nhập KH tháng                             18
Mở cảnh báo nhập KH năm                               18
Mở kết quả liên ngành (tháng n-1)                     17
Mở dữ liệu TH tháng                                   16
Mở dữ liệu TH ngày                                    15
Mở dữ liệu KH quý                                     12
Mở dữ liệu KH tháng                                   12
Mở báo cáo dữ liệu tin nhắn kinh doanh                11
Mở báo cáo VPTD                                       11
Mở dữ liệu KH năm                                     10
Mở báo cáo chốt tháng                                 10
Mở báo cáo chốt quý                                   10
Mở báo cáo tỷ lệ chiết khấu thẻ cào                   10
Mở báo cáo chốt năm            

# Đánh giá với độ chính xác Precision (Tổng số dự đoán đúng / Tổng số dự đoán)

In [18]:
dictionary = {k: v for v, k in enumerate(database)}
preds = torch.tensor(results['Dự đoán myself'].apply(lambda x: dictionary[x]).tolist())
tags = torch.tensor(results['Câu lệnh có sẵn'].apply(lambda x: dictionary[x]).tolist())
precision = Precision(task="multiclass", average='micro', num_classes=len(dictionary))
print(f"Độ chính xác (Precision) của mô hình: {round(precision(preds, tags).item()*100, 2)}")

KeyError: ignored

In [ ]:
len(dictionary)